In [1]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import Box, Histogram, Layout, Figure, Bar, Scatter, Marker, Line
print __version__ # requires version >= 1.9.0
from plotly import tools

init_notebook_mode(connected=True) # run at the start of every ipython notebook

/Users/agraves/Brightmd/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning:

Matplotlib is building the font cache using fc-list. This may take a moment.



1.12.4


In [4]:
import pandas as pd
import csv

HEADERS = ['quarter', 'agegrp', 'gender', 'urban', 'total']
FILE = 'aggregated_amb_unique.csv'
AGEGRP = {'0-4': '00-04', '5-9': '05-09'}

In [3]:
with open(FILE) as f:
    reader = csv.DictReader(f, fieldnames=HEADERS)
    reader.next()
    df = pd.DataFrame([row for row in reader])
    df['total'] = pd.to_numeric(df['total'])

StopIteration: 

In [ ]:
df.head()

## Records by age

In [ ]:
df_per_age = df.groupby(['agegrp']).aggregate('sum').reset_index()
df_per_age['agegrp'] = 'age ' + df_per_age['agegrp'].astype(str)
iplot(Figure(data=[Bar(x=df_per_age['agegrp'], y=df_per_age['total'])]))

## Histogram by age and gender

In [ ]:
df_per_age_gender = df.groupby(['agegrp', 'gender']).aggregate('sum').reset_index()
df_per_age_gender['agegrp'] = 'age ' + df_per_age_gender['agegrp'].astype(str)

female = df_per_age_gender[df_per_age_gender['gender'] == "F"]
male = df_per_age_gender[df_per_age_gender['gender'] == "M"]

age_gender_data = [Bar(x=female['agegrp'], y=female['total'], name="Females"), Bar(x=male['agegrp'], y=male['total'], name="Males")]


layout = Layout(
    barmode='stack'
)

fig = Figure(data=age_gender_data, layout=layout)
iplot(fig)


## Histogram by age and rurality

In [ ]:
df_per_age_rural = df.groupby(['agegrp', 'urban']).aggregate('sum').reset_index()
df_per_age_rural['agegrp'] = 'age ' + df_per_age_gender['agegrp'].astype(str)

urban = df_per_age_rural[df_per_age_rural['urban'] == "1"]
rural = df_per_age_rural[df_per_age_rural['urban'] == "0"]

age_rural = [Bar(x=urban['agegrp'], y=urban['total'], name="Urban"), Bar(x=rural['agegrp'], y=rural['total'], name="Rural")]


layout = Layout(
    barmode='stack'
)

fig = Figure(data=age_rural, layout=layout)
iplot(fig)


## Unique patients per group age and quarter

In [ ]:
df_per_quarter_age = df.groupby(['quarter', 'agegrp']).aggregate('sum')
df_per_quarter_age = df_per_quarter_age.reset_index()
df_per_quarter_age['agegrp'] = df_per_quarter_age['agegrp'].apply(lambda x: AGEGRP[x] if x in AGEGRP else x)
df_per_quarter_age.head(20)

max_value = max(df_per_quarter_age['total'])
ages = list(df_per_quarter_age['agegrp'].unique()) #['Q1', 'Q2', 'Q3', 'Q4']
ages = sorted(ages)
fig = tools.make_subplots(rows=int(round(len(ages)/2.0)), cols=2, subplot_titles=ages, print_grid=False)
for i, q in enumerate(ages):
    z = df_per_quarter_age[df_per_quarter_age['agegrp'] == q]
    fig.append_trace(Scatter(x=z['quarter'], y=z['total'], name=q), int(i/2.0 + 1), i%2 + 1)

fig['layout'].update(height=3000, width=1000, title='Unique visitors per age group per quarter', )

iplot(fig)

In [ ]:
df_per_quarter_age[df_per_quarter_age['agegrp'] == '0-4']['total'].c